In [21]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [ ]:
api_key="pinecone api"

In [1]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [5]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

#intialize the pinecone
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [6]:
index = pc.Index(index_name)
index

In [7]:
!pip install python-dotenv

In [8]:
#vector embeding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import os
os.environ["HF_TOKEN"] = "HF_TOKEN"

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

C:\Users\malah\AppData\Local\Temp\ipykernel_14184\2194665236.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\malah\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malah\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [24]:
sentences = [
    "In 2013,I visited Paris for vacation by the flight Vistara",
    "In 2022,I visited America for client meetings by the flight Air india"
    "In 2023, I visited Japan for sister marriage by the flight Singapore airlines"
]

#tfidf values on these sentences
bm25_encoder.fit(sentences)

#store the values in json format
bm25_encoder.dump("bm25_values.json")

#load to your BM25Encode object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
#retriever
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)
retriever


PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000024732426B10>, index=<pinecone.data.index.Index object at 0x0000024721C16780>)

In [ ]:
retriever.add_texts([
    "In 2013,I visited Paris for vacation by the flight Vistara",
    "In 2022,I visited America for client meetings by the flight Air india"
    "In 2023, I visited Japan for sister marriage by the flight Singapore airlines"
])


  0%|          | 0/1 [00:00<?, ?it/s]

[]

In [27]:
retriever.invoke("what city did i travel for client meeting")

[Document(metadata={'score': 0.319895357}, page_content='In 2022,I visited America for client meetings by the flight Air indiaIn 2023, I visited Japan for sister marriage by the flight Singapore airlines'),
 Document(metadata={'score': 0.164170951}, page_content='In 2013,I visited Paris for vacation by the flight Vistara')]

In [28]:
retriever.invoke("what city did i travel for sister  marriage")

[Document(metadata={'score': 0.317570388}, page_content='In 2022,I visited America for client meetings by the flight Air indiaIn 2023, I visited Japan for sister marriage by the flight Singapore airlines'),
 Document(metadata={'score': 0.143885419}, page_content='In 2013,I visited Paris for vacation by the flight Vistara')]

In [29]:
retriever.invoke("what city did i travel for vacation")

[Document(metadata={'score': 0.357324272}, page_content='In 2013,I visited Paris for vacation by the flight Vistara'),
 Document(metadata={'score': 0.207717955}, page_content='In 2022,I visited America for client meetings by the flight Air indiaIn 2023, I visited Japan for sister marriage by the flight Singapore airlines')]